In [53]:
import numpy as np
import pandas as pd
#import training setvalid_newfeature
a = pd.read_csv('/project/tffm-master/xg_valid_pctr.csv')
#import validation set
b = pd.read_csv('/project/tffm-master/fmv.csv')
c = pd.read_csv('/project/Data/lgbm_valid_pctr.csv')

In [54]:
a.head()

,click,click_pred_per,click_pred_class
0,0,0.000154,0
1,0,0.000162,0
2,0,0.000131,0
3,0,0.000119,0
4,0,0.000108,0


In [55]:
b.head()

,click,click_pred_per,click_pred_class
0,0,8.073824e-06,0
1,0,1.511657e-05,0
2,0,1.075781e-06,0
3,0,6.874001e-07,0
4,0,1.298044e-05,0


In [56]:
z=a['click_pred_per']+b['click_pred_per']
z=z/2

In [18]:
eee=a.copy()

In [41]:
eee['comb']=a['click_pred_per']+b['click_pred_per']+c['click_pred_per']

In [42]:
eee['comb']=eee['comb']/3

In [43]:
eee.to_csv('/project/Data/combine1.csv', encoding='utf-8', index=False)


In [4]:
c.head()

,click,click_pred_per,click_pred_class
0,0,0.000113,0
1,0,0.000081,0
2,0,0.000116,0
3,0,0.000167,0
4,0,0.000067,0


In [5]:
d=a['click_pred_per']+b['click_pred_per']+c['click_pred_per']

In [6]:
d=d/3

In [14]:
d

0         0.000092
1         0.000086
2         0.000083
3         0.000096
4         0.000063
5         0.000224
6         0.000330
7         0.000381
8         0.000194
9         0.000084
10        0.000591
11        0.000174
12        0.000072
13        0.000634
14        0.000309
15        0.000071
16        0.000163
17        0.000135
18        0.001103
19        0.000184
20        0.000262
21        0.000550
22        0.000275
23        0.000092
24        0.000386
25        0.000459
26        0.000049
27        0.000447
28        0.000331
29        0.001081
            ...   
303895    0.004794
303896    0.000071
303897    0.000761
303898    0.000587
303899    0.000329
303900    0.000206
303901    0.000067
303902    0.000265
303903    0.000068
303904    0.000338
303905    0.000359
303906    0.000389
303907    0.000156
303908    0.000142
303909    0.000091
303910    0.000364
303911    0.000348
303912    0.000132
303913    0.000416
303914    0.000050
303915    0.000552
303916    0.

In [57]:
df_valid = pd.read_csv("/project/Data/validation.csv")

In [58]:
# parameters
total_budget = 6250000
payprice = df_valid['payprice'].as_matrix()
Train_df_qr = df_valid.copy()
avgCTR = 1793/2430981 #train set
#tuning...

In [59]:
def generate_bidprice(base_bid, test_predictions):
    avgCTR = 1793/2430981#train set
    bidprices = base_bid*test_predictions/avgCTR
    return bidprices

In [60]:
def Prob3(Train_df,payprice,base_bid,test_predictions):
    Train_Const_summ = pd.DataFrame({'Clicks':'', 'Spend':'', 'CTR':'', 'CPM':'', 'eCPC':''}, index=[0])
    
    #if count_break = 5 break
    count_break = 0
    
    current_budget = total_budget
    Train_df['win'] = 0 #initialize win label
    bidprices =generate_bidprice(base_bid, test_predictions)
    #start biding...
    for i in Train_df.index:
        current_bidprice = bidprices[i]
        #consider budget
        if current_budget <= current_bidprice:              
            #count lack budget times
            count_break = count_break + 1
#             print('lack of budget!',"current_budget:",current_budget,";current_bidprice:",current_bidprice,';count_break:',count_break)
            if count_break > 5:
                break  
            #reset bidprice
            current_bidprice = current_budget
        else:
            count_break = 0
            

        #Winning criterion:
        current_payprice = payprice[i]
        if current_bidprice > current_payprice:
            Train_df.at[i, 'win'] = int(1)
            current_budget = current_budget - current_payprice
        else:
            Train_df.at[i, 'win'] = int(0)

    #biding end        
    
    Train_df_new = Train_df[Train_df['win']!=0]
    #wintimes,_ = Train_df_new.shape
    #print('wintimes:',wintimes)
    #print('current_budget',current_budget)

    
#     #commented in for
    Clk = Train_df_new['click'].sum()
    Imp = Train_df_new['click'].count()
    CTR= Clk/Imp
    Spd = total_budget - current_budget
    CPM = 1000*Spd/Imp
    eCPC = Spd/Clk
    
    Train_Const_summ.loc[0,'Clicks']=Clk
    Train_Const_summ.loc[0,'Spend']=Spd
    Train_Const_summ.loc[0,'CTR']=CTR
    Train_Const_summ.loc[0,'CPM']=CPM
    Train_Const_summ.loc[0,'eCPC']=eCPC

    print(Train_Const_summ)
    
        
    return Train_df,current_budget

In [61]:
def calculate_click(base_bid,Train_df,payprice,test_predictions):
#     print('base_bid:',base_bid)
    result_df,current_budget = Prob3(Train_df,payprice,base_bid,test_predictions)
#     print('current_budget:',current_budget)
    Train_df_new = result_df[result_df['win']!=0]
    wintimes,_ = Train_df_new.shape
#     print('wintimes:',wintimes)
#     print('----------------')
    Clk = Train_df_new['click'].sum()    
    return Clk

In [65]:
#test range
base_bids = np.arange(160,180,1)#区间可以适当左右调整一下，但大概就是这个范围
count = 0
maxClks = []

In [66]:
count = count + 1
print(str(count)+" valid result is being test...")
test_predictions=z
Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
print("max click@",Clks.index(max(Clks)),"base_bid=",base_bids[Clks.index(max(Clks))],"click =",max(Clks))
maxClks.append(max(Clks))
#     fig = plt.figure()
#     plt.plot(base_bids,Clks)
#     mpld3.display(fig)

1 valid result is being test...
       CPM         CTR Clicks    Spend     eCPC
0  47970.5  0.00135922    169  5964463  35292.7
     CPM         CTR Clicks    Spend     eCPC
0  48050  0.00135388    169  5997885  35490.4
       CPM         CTR Clicks    Spend   eCPC
0  48152.7  0.00134795    169  6037191  35723
       CPM         CTR Clicks    Spend     eCPC
0  48226.7  0.00134271    169  6070050  35917.5
       CPM         CTR Clicks    Spend     eCPC
0  48312.6  0.00134521    170  6105454  35914.4
     CPM         CTR Clicks    Spend     eCPC
0  48397  0.00133991    170  6140321  36119.5
       CPM         CTR Clicks    Spend     eCPC
0  48496.1  0.00133444    170  6178116  36341.9
     CPM         CTR Clicks    Spend     eCPC
0  48586  0.00132948    170  6212695  36545.3
       CPM         CTR Clicks    Spend     eCPC
0  48665.4  0.00132437    170  6246840  36746.1
       CPM         CTR Clicks    Spend     eCPC
0  48748.4  0.00131816    169  6249984  36982.2
       CPM         CTR C

In [20]:
count = count + 1
print(str(count)+" valid result is being test...")
test_predictions=c
Clks = [calculate_click(base_bid,Train_df_qr,payprice,test_predictions) for base_bid in base_bids]
print("max click@",Clks.index(max(Clks)),"base_bid=",base_bids[Clks.index(max(Clks))],"click =",max(Clks))
maxClks.append(max(Clks))
#     fig = plt.figure()
#     plt.plot(base_bids,Clks)
#     mpld3.display(fig)

1 valid result is being test...
       CPM         CTR Clicks    Spend     eCPC
0  48815.5  0.00131216    168  6249999  37202.4
       CPM         CTR Clicks    Spend     eCPC
0  48887.6  0.00130628    167  6249987  37425.1
       CPM       CTR Clicks    Spend     eCPC
0  48977.2  0.001293    165  6249980  37878.7
     CPM        CTR Clicks    Spend     eCPC
0  49070  0.0012876    164  6249994  38109.7
       CPM         CTR Clicks    Spend     eCPC
0  49139.5  0.00128156    163  6249998  38343.5
       CPM        CTR Clicks    Spend     eCPC
0  49238.7  0.0012684    161  6249973  38819.7
       CPM         CTR Clicks    Spend     eCPC
0  49322.8  0.00124688    158  6249993  39556.9
       CPM         CTR Clicks    Spend     eCPC
0  49416.7  0.00122554    155  6249981  40322.5
       CPM         CTR Clicks    Spend     eCPC
0  49508.9  0.00121198    153  6249999  40849.7
       CPM         CTR Clicks    Spend     eCPC
0  49586.2  0.00121387    153  6249994  40849.6
max click@ 0 base_bi

In [ ]:
153

In [23]:
e=pd.read_csv('/project/tffm-master/fm.csv')

In [24]:
f=pd.read_csv('/project/tffm-master/xg_test_pctr.csv')

In [30]:
g=pd.read_csv('/project/Data/pctr_lgbm_test.csv')

In [31]:
e

,click,click_pred_per,click_pred_class
0,NaN,1.384021e-04,0
1,NaN,2.382462e-03,0
2,NaN,3.662250e-06,0
3,NaN,4.726222e-05,0
4,NaN,1.100558e-03,0
5,NaN,5.148010e-06,0
6,NaN,5.402221e-05,0
7,NaN,1.213808e-03,0
8,NaN,5.014401e-05,0
9,NaN,4.903528e-04,0


In [32]:
f

,Unnamed: 0,bidprice,click_pred_per,click_pred_class
0,0,106.831530,0.000601,0
1,1,274.341351,0.001545,0
2,2,23.719198,0.000134,0
3,3,113.739314,0.000640,0
4,4,86.940014,0.000489,0
5,5,120.532011,0.000679,0
6,6,29.839475,0.000168,0
7,7,76.893864,0.000433,0
8,8,62.385191,0.000351,0
9,9,85.046060,0.000479,0


In [33]:
g

,bidprice,click_pred_per,click_pred_class
0,244.080088,0.001385,0
1,335.697919,0.001905,0
2,16.689485,0.000095,0
3,106.045246,0.000602,0
4,106.171178,0.000602,0
5,103.802702,0.000589,0
6,26.574267,0.000151,0
7,101.980506,0.000579,0
8,44.158626,0.000251,0
9,51.102187,0.000290,0


In [34]:
h=e['click_pred_per']+f['click_pred_per']+g['click_pred_per']

In [35]:
h=h/3

In [47]:
h

0         0.000708
1         0.001944
2         0.000077
3         0.000430
4         0.000731
5         0.000424
6         0.000124
7         0.000742
8         0.000217
9         0.000420
10        0.000236
11        0.002540
12        0.000401
13        0.000222
14        0.000438
15        0.000176
16        0.000205
17        0.000162
18        0.000340
19        0.000426
20        0.000104
21        0.000514
22        0.000206
23        0.000329
24        0.000053
25        0.000732
26        0.000183
27        0.000553
28        0.000453
29        0.000069
            ...   
303345    0.000308
303346    0.000640
303347    0.000184
303348    0.000195
303349    0.000548
303350    0.000088
303351    0.001273
303352    0.000930
303353    0.002612
303354    0.000185
303355    0.000088
303356    0.001110
303357    0.000082
303358    0.000194
303359    0.000085
303360    0.000196
303361    0.000164
303362    0.000344
303363    0.000441
303364    0.000230
303365    0.000243
303366    0.

In [50]:
#Test_df = pd.read_csv("/project/Data/mytestset.csv")
Group=  pd.read_csv('/project/Data/Group_xx.csv')

In [37]:
base_bid=153
#这个变成你测出来最高clk的basebid！！！！！！！
avgCTR = 1793/2430981#train set
v = base_bid*h/avgCTR


In [38]:
v

0         146.915443
1         403.240886
2          16.034837
3          89.150583
4         151.598598
5          88.003242
6          25.777642
7         153.874305
8          45.078426
9          87.063593
10         49.030179
11        526.813888
12         83.122772
13         46.013726
14         90.853076
15         36.446307
16         42.456750
17         33.600922
18         70.603505
19         88.322714
20         21.610955
21        106.643496
22         42.714776
23         68.158271
24         11.041085
25        151.762401
26         38.047282
27        114.686942
28         93.870131
29         14.219637
             ...    
303345     63.965632
303346    132.715887
303347     38.082039
303348     40.403420
303349    113.720788
303350     18.302651
303351    264.002100
303352    192.870023
303353    541.796467
303354     38.405729
303355     18.217024
303356    230.288232
303357     16.977514
303358     40.293644
303359     17.733927
303360     40.742830
303361     34

In [51]:
Group['bidprice']=bp
Group.head()


,bidid,bidprice
0,366c563de7d90feb9d4dab53e795a93fb3157387,147.551663
1,29167d4caa719788b5a342dbaa25151d53121f80,325.963273
2,ff8bc3f4d44a3ea60c5f3a3a8fbe7cd98fb2966e,17.162003
3,844c2da00d45315f20b748ec131c26ee99a7cbc7,93.038319
4,c6017f0ad0c44d7d0c9b62583ea863f28941c0ca,151.712928


In [52]:
Group.to_csv('/project/Data/Group_666666.csv', encoding='utf-8', index=False)


In [45]:
lam = 0.0000030
c = 330

In [46]:
def generate_bidprice(c, lam, pCTR):
#     avgCTR = 1793/2430981#train set
    z = ((pCTR+((c**2)*(lam**2)+(pCTR**2))**(1/2))/(c*lam))**(1/3)
    bidprices = c*(z-1/z)
    assert bidprices.shape == pCTR.shape
    return bidprices

In [48]:
bp=generate_bidprice(c,lam,h)

In [49]:
bp

0         147.551663
1         325.963273
2          17.162003
3          93.038319
4         151.712928
5          91.898699
6          27.550541
7         153.720784
8          47.953180
9          90.963854
10         52.091394
11        386.975661
12         87.028479
13         48.933992
14         94.725570
15         38.863787
16         45.199554
17         35.854233
18         74.375366
19         92.216226
20         23.113152
21        110.151907
22         45.470847
23         71.878506
24         11.822812
25        151.857762
26         40.554364
27        117.849539
28         97.704378
29         15.222128
             ...    
303345     67.579118
303346    134.693390
303347     40.591044
303348     43.038538
303349    116.930798
303350     19.583903
303351    240.327375
303352    186.695562
303353    393.665970
303354     40.932589
303355     19.492495
303356    215.896072
303357     18.168968
303358     42.922901
303359     18.976724
303360     43.396003
303361     36